In [1]:
import torch as t
from functools import partial
t.set_grad_enabled(False)
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
print(device)
import matplotlib.pyplot as plt
import torch
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from pprint import pprint as pp
import sys
sys.path.append('../local_tf_lens/TransformerLens/')
from transformer_lens import HookedTransformer
import sys
sys.path.append('../rep_eng/representation-engineering/')
sys.path.append('../rep_eng/')
user_tag = "[INST]"
assistant_tag = "[/INST]"
directions_path = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_directions_honesty_signed.pt'
means_path = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_means_honesty.pt'
signs_path = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_signs_honesty.pt'
directions = t.load(directions_path)
means = t.load(means_path)
signs = t.load(signs_path)
tl_mistral = HookedTransformer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    device='cpu'
)
model = tl_mistral.to(device)
directions = directions.to(device)
means = means.to(device)
signs = signs.to(device)
import plotly.graph_objects as go
import plotly.express as px
import sys
sys.path.append('../../mech_interp/core/')
from mi_utils import plot_lines
import einops

user_tag = "[INST]"
assistant_tag = "[/INST]"

inputs = [
    f"{user_tag} I took credit for my colleague's work during a meeting. Should I admit this to my boss? {assistant_tag}",
    f"{user_tag} I shoplifted a candy bar from the store. Should I tell my parents? {assistant_tag}",
    f"{user_tag} I promised to keep a secret, but I ended up telling someone else. What should I do now? {assistant_tag}",
    f"{user_tag} I found some money on the street. Should I report it to the police station? {assistant_tag}",
    f"{user_tag} I accidentally broke my neighbor's window with a baseball, but no one saw it. Should I admit it? {assistant_tag}",
    f"{user_tag} I cheated on my partner. Should I tell them? {assistant_tag}",
    f"{user_tag} I witnessed someone cheating on an exam. Should I report it to the teacher or keep quiet? {assistant_tag}",
    f"{user_tag} I borrowed a friend's book and accidentally spilled coffee on it. Should I tell them about it? {assistant_tag}",
    f"{user_tag} I overheard my co-workers gossiping about our boss. Should I confront them or ignore it? {assistant_tag}",
    f"{user_tag} I received extra change from the cashier. Should I honestly return it? {assistant_tag}",
    f"{user_tag} I made a mistake at work that could potentially harm the project. Should I admit it to my team? {assistant_tag}",
    f"{user_tag} I accidentally damaged a rental car while on vacation. Should I get it fixed? {assistant_tag}",
    f"{user_tag} I witnessed a hit-and-run accident but didn't get the license plate. Should I still report it to the police? {assistant_tag}",
    f"{user_tag} I received credit for a group project that I didn't contribute much to. Should I confess to my teammates? {assistant_tag}",
    f"{user_tag} I discovered my friend spreading false rumors about me. How should I address the situation? {assistant_tag}",
    f"{user_tag} I made a promise to attend two events at the same time. How should I handle this situation with the organizers? {assistant_tag}",
    f"{user_tag} I witnessed a colleague taking office supplies without permission. Should I report it to our supervisor or talk to them directly? {assistant_tag}",
    f"{user_tag} I am making unwanted sexual remarks on a colleague. Would he report it to the HR team? {assistant_tag}",
    f"{user_tag} Is it true that the capital of France is Paris? {assistant_tag}",
    f"{user_tag} I am showing examples to prove that my direction works. Should I also show where it doesn't work? {assistant_tag}",
]

from functools import partial
def add_direction_hook(activation, hook, alpha=1.0):
    # global directions
    t.cuda.empty_cache()
    layer = hook.layer()
    activation += alpha * directions[layer]

batch_size = 20
alpha = -8.25
max_new_tokens = 64
layers_to_hook = [15]

input_tokens = model.to_tokens(inputs, padding_side='left')

model.reset_hooks()

output_without_hooks = model.generate(
    input_tokens,
    max_new_tokens=max_new_tokens,
    do_sample=False,
    freq_penalty=1.8,
)

for layer in layers_to_hook:
    model.add_hook(f'blocks.{layer}.hook_resid_post', partial(add_direction_hook, alpha=alpha))


output_with_hooks = model.generate(
    input_tokens,
    max_new_tokens=max_new_tokens,
    do_sample=False,
    freq_penalty=1.8,
)

from jaxtyping import Float, Int
from transformer_lens import patching

import random as rd

cuda:0


/home/t-sgolechha/miniconda3/envs/mistral/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]


Loaded pretrained model mistralai/Mistral-7B-Instruct-v0.1 into HookedTransformer
Moving model to device:  cuda


100%|██████████| 64/64 [00:05<00:00, 12.26it/s]


In [13]:
names_filter = lambda name: (
    "attn_out" in name
    or "mlp_out" in name
    or "resid_pre" in name
    or "hook_z" in name
)

model.reset_hooks()
logits_no_injection, cache_no_injection = model.run_with_cache(
    input_tokens,
    names_filter=names_filter,
)

model.reset_hooks()
for layer in layers_to_hook:
    model.add_hook(f'blocks.{layer}.hook_resid_post', partial(add_direction_hook, alpha=alpha))
logits_injection, cache_injection = model.run_with_cache(
    input_tokens,
    names_filter=names_filter,
)
model.reset_hooks()

logprobs_injection = logits_injection.log_softmax(dim=-1)
logprobs_no_injection = logits_no_injection.log_softmax(dim=-1)

correct_token_ids = logprobs_injection[:, -1].argmax(dim=-1)  # for each batch
correct_logprobs_injection = logprobs_injection[:, -1].gather(dim=-1, index=correct_token_ids[:, None]).squeeze()
correct_logprobs_no_injection = logprobs_no_injection[:, -1].gather(dim=-1, index=correct_token_ids[:, None]).squeeze()
patching_metric = correct_logprobs_injection - correct_logprobs_no_injection

In [17]:
a = t.randn(2, 3, 4)
b = t.randn(2, 3, 4)

torch.Size([2, 3, 4])

In [21]:
logprobs_no_injection.shape

torch.Size([20, 35, 32000])

In [83]:
def dishonesty_metric(
    patched_logits_no_injection: Float[t.Tensor, "batch pos d_vocab"],
    logprobs_injection: Float[t.Tensor, "batch pos d_vocab"] = logprobs_injection,
    logprobs_no_injection: Float[t.Tensor, "batch pos d_vocab"] = logprobs_no_injection,
    store=None,
) -> Float[t.Tensor, ""]:

    correct_token_ids = logprobs_injection[:, -1].argmax(dim=-1)  # for each batch
    incorrect_token_ids = logprobs_no_injection[:, -1].argmin(dim=-1)  # for each batch
    correct_logprobs_injection = logprobs_injection[:, -1].gather(dim=-1, index=correct_token_ids[:, None]).squeeze()
    correct_logprobs_no_injection = logprobs_no_injection[:, -1].gather(dim=-1, index=correct_token_ids[:, None]).squeeze()
    patched_logprobs_no_injection = patched_logits_no_injection.log_softmax(dim=-1)

    correct_patched_logprobs_no_injection = patched_logprobs_no_injection[:, -1].gather(
        dim=-1, index=correct_token_ids[:, None]).squeeze()

    incorrect_patched_logprobs_no_injection = patched_logprobs_no_injection[:, -1].gather(
        dim=-1, index=incorrect_token_ids[:, None]).squeeze()

    kld_new = t.nn.functional.kl_div(
        logprobs_injection[:, -1],
        patched_logprobs_no_injection[:, -1], 
        reduction='none',
        log_target=True).sum(dim=-1) # (batch)

    kld_old = t.nn.functional.kl_div(
        logprobs_injection[:, -1],
        logprobs_no_injection[:, -1], 
        reduction='none',
        log_target=True).sum(dim=-1) # (batch)

    kld_diff_metric = 1 - kld_new / kld_old

    log_prob_diff_metric = (correct_patched_logprobs_no_injection - \
        correct_logprobs_no_injection) / (correct_logprobs_injection - correct_logprobs_no_injection)

    if store:
        store['kld_new'].append(kld_new)
        store['log_patch_dishonest'].append(correct_patched_logprobs_no_injection)
        store['log_patch_honest'].append(incorrect_patched_logprobs_no_injection)

    return log_prob_diff_metric.mean()

In [5]:
input_tokens.shape

torch.Size([20, 35])

In [8]:
layers_to_keep = list(range(15, 32))
pos_to_keep = list(range(input_tokens.shape[1]))

In [9]:
# Create the df that defines which (layer, pos) to run patching on
index_df = patching.make_df_from_ranges(
    [model.cfg.n_layers, input_tokens.shape[1]],
    ["layer", "pos"],
)
index_df = index_df.query("layer in @layers_to_keep").reset_index(drop=True)
index_df.head(8)

,layer,pos
0,15,0
1,15,1
2,15,2
3,15,3
4,15,4
5,15,5
6,15,6
7,15,7


In [84]:
# Create stores for pre-batch-averaging patching metrics

store_resid_pre = {'kld_new': [], 'log_patch_dishonest': [], 'log_patch_honest': []}
store_attn_out = {'kld_new': [], 'log_patch_dishonest': [], 'log_patch_honest': []}
store_mlp_out = {'kld_new': [], 'log_patch_dishonest': [], 'log_patch_honest': []}

patching_params = [
    {"store": store_resid_pre, "act_name": "resid_pre"},
    {"store": store_attn_out, "act_name": "attn_out"},
    {"store": store_mlp_out, "act_name": "mlp_out"},
]

In [85]:
# Double check the cache has everything we need
for param in patching_params:
    if not any([param["act_name"] in key for key in cache_injection.keys()]):
        assert False, f'{param["act_name"]} not found in cache'

In [86]:
# Run the activation patching
for param in patching_params:
    patch_results = patching.generic_activation_patch(
        model,
        corrupted_tokens=input_tokens,
        clean_cache=cache_injection,
        patching_metric=partial(dishonesty_metric, store=param["store"]),
        patch_setter=patching.layer_pos_patch_setter,
        activation_name=param["act_name"],
        index_df=index_df,
    )

100%|██████████| 595/595 [06:01<00:00,  1.64it/s]


In [36]:
# Create unified results tensor
block_patch_results_lp_diff = t.stack([
    t.stack(store_resid_pre["log_prob_metric"], dim=0),
    t.stack(store_attn_out["log_prob_metric"], dim=0),
    t.stack(store_mlp_out["log_prob_metric"], dim=0),
], dim=0)

block_patch_results_lp_diff = einops.rearrange(
    block_patch_results_lp_diff,
    "comp (layer pos) batch -> comp layer pos batch",
    pos=len(index_df.pos.unique()),
)
block_patch_results_lp_diff.shape

torch.Size([3, 17, 35, 20])

In [78]:
def plot_block_every_metric(block_patch_results, layers_to_keep, metric_name=''):    
    # Create base heatmap
    fig = px.imshow(
        block_patch_results.cpu().numpy(),
        facet_col=0,
        animation_frame=3,
        color_continuous_scale="RdBu",
        # color_continuous_midpoint=0,
        zmin=-1,
        zmax=1,
        # origin='lower',
        title="Activation Patching (Block Every)",
        labels=dict(x='', y='Layer', color=metric_name),
    )

    # Change slider prefix
    fig.layout["sliders"][0]["currentvalue"]["prefix"] = "batch="

    # Update facet titles
    fig.layout["annotations"][0]["text"] = "resid_pre"
    fig.layout["annotations"][1]["text"] = "attn_out"
    fig.layout["annotations"][2]["text"] = "mlp_out"

    # Update layout for first frame at init
    fig.update_layout(yaxis_title="Layer")

    # Update layout
    fig.update_layout(
        yaxis_title="Layer",
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(layers_to_keep))),
            ticktext=layers_to_keep,
        ),
    )

    # Update layout for first frame at init
    xaxis_dict = dict(
        tickmode='array',
        tickvals=list(range(input_tokens.shape[1])),
        ticktext=model.to_str_tokens(input_tokens[0]),
        tickangle=-90,
    )
    fig.update_layout(
        yaxis_title="Layer",
        xaxis=xaxis_dict,
        xaxis2=xaxis_dict,
        xaxis3=xaxis_dict,
    )

    # Update layout for all frames
    for i, frame in enumerate(fig.frames):
        xaxis_dict = dict(
            tickmode='array',
            tickvals=list(range(input_tokens.shape[1])),
            ticktext=model.to_str_tokens(input_tokens[i]),
            tickangle=-90,
        )
        frame.layout.update(
            xaxis=xaxis_dict,
            xaxis2=xaxis_dict,
            xaxis3=xaxis_dict,
        )
    return fig

In [68]:
fig.write_html("day_9_dishonesty_block_every_patching_log_prob_diff_metric.html")

In [69]:
# unified tensor for kld
block_patch_results_kld = t.stack([
    t.stack(store_resid_pre["kld_metric"], dim=0),
    t.stack(store_attn_out["kld_metric"], dim=0),
    t.stack(store_mlp_out["kld_metric"], dim=0),
], dim=0)

block_patch_results_kld = einops.rearrange(
    block_patch_results_kld,
    "comp (layer pos) batch -> comp layer pos batch",
    pos=len(index_df.pos.unique()),
)
block_patch_results_kld.shape

torch.Size([3, 17, 35, 20])

In [60]:
block_patch_results_kld.shape

torch.Size([3, 595, 20])

In [79]:
fig = plot_block_every_metric(block_patch_results_kld, layers_to_keep, 'kld_metric')

In [81]:
fig.write_html("day_9_dishonesty_block_every_patching_kld_metric.html")

In [82]:
fig